## Load packages and sample data

In [4]:
import re
import pandas as pd
from typing import List
import matplotlib.pyplot as plt
import sys, os
sys.path.append("\\".join(os.getcwd().split("\\")[:-1]))
%load_ext autoreload
%autoreload 2
from scraper import utils

c:\Users\ma1021525\Anaconda3\envs\geoharvester\lib\site-packages\torch\__init__.py:749: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)


In [ ]:
url_github_repo = "https://github.com/FHNW-IVGI/Geoharvester/blob/main/scraper/data/"
url_github_repo_suffix = "?raw=true"
url_geoservices_CH_csv = "{}geoservices_CH.csv{}".format(url_github_repo,url_github_repo_suffix)
fields_to_include = ["provider","title", "keywords", "abstract", "service", "endpoint", "preview"]

def load_data(rawdata=False):
    if rawdata:
        dataframe = pd.read_csv(url_geoservices_CH_csv, low_memory=False)
    else:
        dataframe = pd.read_csv(url_geoservices_CH_csv, usecols=fields_to_include, low_memory=False)
    return dataframe

# dataframe_some_cols = load_data()
# database = dataframe_some_cols.fillna("nan")
dataframe_some_cols = load_data(rawdata=True)
database_raw = dataframe_some_cols.fillna("nan")

In [ ]:
db = pd.read_csv("./data/geoservices_CH.csv", low_memory=False)
def count(row, exc=0):
    try:
        num = len(str(row).split())
    except TypeError:
        num = exc
    except AttributeError:
        num = exc
    if pd.isna(row):
        num = 0
    return num
db['abstract_w_count'] = db['ABSTRACT'].apply(count)
db['keywords_w_count'] = db['KEYWORDS'].apply(count)
db.replace(['Bund','FL_LI','Geodienste','KT_AG','KT_AI','KT_AR','KT_BE','KT_BL','KT_BS','KT_FR','KT_GE','KT_GL','KT_GR','KT_JU','KT_SG','KT_SH','KT_SO','KT_SZ','KT_TG','KT_TI','KT_UR','KT_VD','KT_ZG','KT_ZH'],
              ['Bund','LI','Geodienste','AG','AI','AR','BE','BL','BS','FR','GE','GL','GR','JU','SG','SH','SO','SZ','TG','TI','UR','VD','ZG','ZH'], inplace=True)
db_nan = db[db['OWNER'] == 'Bund'].isna().sum()

In [ ]:
db[(db['OWNER'] == 'AI') & (db['keywords_w_count'] == 1)]

In [ ]:
for column in db.columns.to_list():
    if (db[column][db[column].isna()]).empty:
        print(f"No field missing in {column}")

In [ ]:
db_gr = db.groupby(['OWNER']).count()[['TITLE','ABSTRACT', 'KEYWORDS', 'NAME']]
db_gr['abstract_perc'] = 100 / db_gr['TITLE'] * db_gr['ABSTRACT']
db_gr['keywords_perc'] = 100 / db_gr['TITLE'] * db_gr['KEYWORDS']
db_gr['name_perc'] = 100 / db_gr['TITLE'] * db_gr['NAME']
db_gr.rename({'Geodienste':'Geodnst'}, inplace=True)
db_gr.sort_values(by=['OWNER'], ascending=False, inplace=True)
db_gr

In [ ]:
plt.figure(figsize = (5,5))
plt.barh(db_gr.index, db_gr['TITLE'])
plt.title("Number of OWS")
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.ylabel("Providers")
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
plt.barh(db_gr.index, db_gr['abstract_perc'])
plt.title("Abstact filled %", fontsize=22)
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.rcParams.update({"font.size":18})
plt.xlim(0,100)
# plt.ylabel("Providers")
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
plt.barh(db_gr.index, db_gr['keywords_perc'], )
plt.title("Keywords filled %", fontsize=22)
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.rcParams.update({"font.size":18})
plt.xlim(0,100)
plt.ylabel("Providers", fontsize=19)
plt.show()

In [ ]:
100 / db['abstract_w_count'].count() * db[db['abstract_w_count'] > 20]['ABSTRACT'].count()

In [ ]:
db['abstract_w_count'].max()

In [ ]:
test = db[['ABSTRACT', 'KEYWORDS', 'OWNER', 'abstract_w_count','keywords_w_count', 'TITLE']]
test = test.groupby(['OWNER']).mean()[['abstract_w_count','keywords_w_count']]
test.rename({'Geodienste':'Geodnst'}, inplace=True)
test.sort_values(by=['OWNER'], ascending=False, inplace=True)

In [ ]:
test1 = test.groupby(['OWNER','keywords_w_count']).count()['TITLE']

In [ ]:
plt.barh(test.index, test['abstract_w_count'], )
plt.title("Abstract Wordcount Median")
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.show()

In [ ]:
plt.figure(figsize = (10,10))
# plt.vlines(x=1.0, ymin=0, ymax=len(test.index.to_list())-0.5, color='grey')
plt.barh(test.index, test['keywords_w_count'], )
plt.title("Keywords average word count", fontsize=22)
plt.autoscale(tight=True)

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.rcParams.update({"font.size":18})
plt.tick_params(axis='y', which='both', left=False)
plt.show()

## Testing functions from utils.py

### Improvement of the TF-IDF with BM25 to execute queries on the text

In [ ]:
# abstracts = result['abstract'].values.tolist()
bm25 = utils.TFIDF_BM25()
bm25.cleansing_ranking(database, column='abstract') # 1421 lines in 36 s

In [ ]:
# fit the vector to the database
bm25.fit()
# search the best match in the vector
res = bm25.search('Brandmeldeanlage')

### Implement the keyword extraction with spacy (NLP)

In [ ]:
NLP = utils.NLP_spacy()
# Keyword
keywords_NLP = NLP.extract_keywords(database, column='abstract') # with small models

In [ ]:
keywords_NLP[0]

In [ ]:
# Original abstract
database['abstract'].values.tolist()[0]

### Test LSA and LSI with gensim for topic modelling

In [ ]:
LSA = utils.LSI_LDA()
abstracts = LSA.preprocess(database)
LSA.compute_coherence_values_LSI((4,16,2))

In [ ]:
best_number_topics = 12
number_of_words = 20
lsamodel = LSA.create_gensim_lsa_model(best_number_topics, number_of_words)

In [ ]:
lsamodel.print_topics(best_number_topics, number_of_words)

### Test Text Summarization and Latent Dirchlet allocation (LDA) for topic modeling

In [ ]:
# https://towardsdatascience.com/text-summarization-for-clustering-documents-2e074da6437a
# https://towardsdatascience.com/nlp-topic-modeling-to-identify-clusters-ca207244d04f

# https://medium.com/plain-simple-software/build-an-ai-text-summarizer-in-python-6209fb23875d

#### Keywords extraction

In [ ]:
NLP = utils.NLP_spacy()
keywords_dataset = NLP.extract_refined_keywords(database, use_rake=True, column='abstract',
                                          keyword_length=3, num_keywords=15)

In [ ]:
keywords_dataset[3]

In [ ]:
database.loc[3]['abstract']

In [ ]:
# Add the extracted keywords to the raw data dataframe
def join_keywords(keywords_list):
    keywords = ', '.join(kw for kw in keywords_list)
    return keywords
database_raw['keywords_nlp'] = list(map(join_keywords, keywords_dataset))

#### GPT model

In [ ]:
import streamlit as st
st.title("Summarizer")
input_text = st.text_area(label='Enter full text:', value='', height=250)
st.button("submit")
output_text = st.text_area(label='Summarized text:', value='', height=250)

In [ ]:
import openai
import os
openai.api_key = os.getenv('OPENAI_KEY') # WARNING: limits to 18$ free credits!

def summarize_GPT(text):
    prompt = f"summarize this text: {text}"
    model = openai.Completion.create(model='text-davinci-003', prompt=prompt, temperature=.5, max_tokens=1000,)
    return model["choices"][0]['text']

In [ ]:
summarize_GPT('''Thermal infrared imagery is very much gaining in importance in the diagnosis of energy
losses in cultural heritage through non-destructive measurement methods. Hence, owing to the fact
that it is a very innovative and, above all, safe solution, it is possible to determine the condition of the
building, locate places exposed to thermal escape, and plan actions to improve the condition of the
facility. The presented work is devoted to the technology of creating a dense point cloud and a 3D
model, based on data obtained from UAV. It has been shown that it is possible to build a 3D point
model based on thermograms with the specified accuracy by using thermal measurement marks and
the dense matching method. The results achieved in this way were compared and, as the result of
this work, the model obtained from color photos was integrated with the point cloud created on the
basis of the thermal images. The discussed approach exploits measurement data obtained with three
independent devices (tools/appliances): a Matrice 300 RTK drone (courtesy of NaviGate); a Phantom
4 PRO drone; and a KT-165 thermal imaging camera. A stone church located in the southern part of
Poland was chosen as the measuring object.''')

#### T5 abstractive summarization

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
model = T5ForConditionalGeneration.from_pretrained("google/mt5-large") #  google/mt5-large
tokenizer = T5Tokenizer.from_pretrained('google/mt5-large')
device = torch.device('cpu')
task_prefix = 'Summarize:'
body = '''Thermal infrared imagery is very much gaining in importance in the diagnosis of energy
losses in cultural heritage through non-destructive measurement methods. Hence, owing to the fact
that it is a very innovative and, above all, safe solution, it is possible to determine the condition of the
building, locate places exposed to thermal escape, and plan actions to improve the condition of the
facility. The presented work is devoted to the technology of creating a dense point cloud and a 3D
model, based on data obtained from UAV. It has been shown that it is possible to build a 3D point
model based on thermograms with the specified accuracy by using thermal measurement marks and
the dense matching method. The results achieved in this way were compared and, as the result of
this work, the model obtained from color photos was integrated with the point cloud created on the
basis of the thermal images. The discussed approach exploits measurement data obtained with three
independent devices (tools/appliances): a Matrice 300 RTK drone (courtesy of NaviGate); a Phantom
4 PRO drone; and a KT-165 thermal imaging camera. A stone church located in the southern part of
Poland was chosen as the measuring object.'''
tokenized_text = tokenizer.encode(task_prefix + body.strip().replace('\n',''), return_tensors="pt").to(device)
summary = model.generate(tokenized_text, num_beams=1, no_repeat_ngram_size=1, min_length=20, max_length=80,
                         early_stopping=True)
output = tokenizer.decode(summary[0], skip_special_tokens=True)


In [ ]:
output

#### Abstractive summarization with SBert model

In [ ]:
# Using Sentence-Bert to summarize the abstract (extractive summary)
# https://www.sbert.net/docs/pretrained_models.html#multi-lingual-models
from summarizer.sbert import SBertSummarizer
model = SBertSummarizer(model='paraphrase-multilingual-MiniLM-L12-v2') #all-MiniLM-L12-v2
body = '''Thermal infrared imagery is very much gaining in importance in the diagnosis of energy
losses in cultural heritage through non-destructive measurement methods. Hence, owing to the fact
that it is a very innovative and, above all, safe solution, it is possible to determine the condition of the
building, locate places exposed to thermal escape, and plan actions to improve the condition of the
facility. The presented work is devoted to the technology of creating a dense point cloud and a 3D
model, based on data obtained from UAV. It has been shown that it is possible to build a 3D point
model based on thermograms with the specified accuracy by using thermal measurement marks and
the dense matching method. The results achieved in this way were compared and, as the result of
this work, the model obtained from color photos was integrated with the point cloud created on the
basis of the thermal images. The discussed approach exploits measurement data obtained with three
independent devices (tools/appliances): a Matrice 300 RTK drone (courtesy of NaviGate); a Phantom
4 PRO drone; and a KT-165 thermal imaging camera. A stone church located in the southern part of
Poland was chosen as the measuring object.'''
summary = model(body, num_sentences=3)
print(summary)

In [ ]:
# with paraphrase-multilingual v2
'''Thermal infrared imagery is very much gaining in importance in the diagnosis of energy
losses in cultural heritage through non-destructive measurement methods. The results achieved in this way were compared and, as the result of
this work, the model obtained from color photos was integrated with the point cloud created on the
basis of the thermal images. A stone church located in the southern part of
Poland was chosen as the measuring object.'''
# with all-MiniLM-L12-v2
'''Thermal infrared imagery is very much gaining in importance in the diagnosis of energy
losses in cultural heritage through non-destructive measurement methods. The results achieved in this way were compared and, as the result of
this work, the model obtained from color photos was integrated with the point cloud created on the
basis of the thermal images. The discussed approach exploits measurement data obtained with three
independent devices (tools/appliances): a Matrice 300 RTK drone (courtesy of NaviGate); a Phantom
4 PRO drone; and a KT-165 thermal imaging camera.'''
# all-mpnet-base-v2
'''Thermal infrared imagery is very much gaining in importance in the diagnosis of energy
losses in cultural heritage through non-destructive measurement methods. The results achieved in this way were compared and, as the result of
this work, the model obtained from color photos was integrated with the point cloud created on the
basis of the thermal images. A stone church located in the southern part of
Poland was chosen as the measuring object.'''
# paraphrase-multilingual-MiniLM-L12-v2
'''Thermal infrared imagery is very much gaining in importance in the diagnosis of energy
losses in cultural heritage through non-destructive measurement methods. It has been shown that it is possible to build a 3D point
model based on thermograms with the specified accuracy by using thermal measurement marks and
the dense matching method. A stone church located in the southern part of
Poland was chosen as the measuring object.'''


#### Latent Dirichlet Allocation (LDA) with gensim for topic modelling

In [ ]:
LDA = utils.LSI_LDA()
texts_tokenized = LDA.preprocess(database, column='abstract') # ca. 5 min
main_topics_lda = LDA.create_gensim_lda_model(categories='eCH') # ca. 3 min
vis = LDA.prepare_plot_lda()

In [ ]:
import pyLDAvis
pyLDAvis.display(vis)

### Topic classification with TF-IDF vectors and predefined categories

#### INSPIRE categories

In [ ]:
import pandas as pd
df = pd.read_csv("data/INSPIRE_categories.csv", sep=';', encoding='UTF-16')
df.columns

In [ ]:
len(df)

In [ ]:
df.loc[0][8]

#### eCH categories

In [ ]:
import pandas as pd
df = pd.read_csv("data/eCH_categories.csv", sep=';', encoding='UTF-16')
df.columns

In [ ]:
len(df)

#### Topic classification with TF-IDF vector

In [ ]:
bm25 = utils.TFIDF_BM25()
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
# prepare the categories for INSPIRE
cats_index = df.index.values
# translate from english to german because the german description is too short
full_desc_DE = [utils.translate(element, lang='de', translator='google') for element in df['complete_description'].tolist()]
description_title_DE = [df['category_DE'].values.tolist()[i]+', '+full_desc_DE[i]
                     for i in range(0, len(df))]
stemmed_categories = [utils.tokenize_abstract(text, output_scores=False, stem_words=True) for
                       text in description_title_DE]
joined_categories = [' '.join(words) for words in stemmed_categories]
vectorizer = bm25.vectorizer
vectorizer.fit(joined_categories)
score_categories = super(TfidfVectorizer, vectorizer).transform(joined_categories)
doc_length = score_categories.sum(1).A1
avd = score_categories.sum(1).mean()

In [ ]:
stemmed_keywords = [utils.stemming_sentence(text) for text in [' '.join(words) for words in keywords_dataset]]

In [ ]:
# prepare the database for the search (keywordds from rake in class NLP_spacy)
# WARNING: the results aren't satisfactory
import numpy as np
dataset = 20
k1 = 1.6
b = 0.75
for word in stemmed_keywords[dataset]:
    query, = super(TfidfVectorizer, vectorizer).transform([word])
    assert sparse.isspmatrix_csr(query)
    kw = score_categories.tocsc()[:, query.indices]
    idf = vectorizer._tfidf.idf_[None, query.indices] - 1.
    scores = (kw.multiply(np.broadcast_to(idf, kw.shape)) * (k1 + 1)/ kw + (k1 * (1 - b + b * doc_length / avd))[:, None]).sum(1).A1
    scores = [round(score, 2) for score in scores if score != 0.0 and not np.isnan(score)]
    category = [cats_index[i] for i in range(0, len(scores)) if scores[i] > 0.0]
    print(category, scores, word)


In [ ]:
description_title_DE[17], description_title_DE[19], description_title_DE[5]

In [ ]:
database.loc[dataset]['abstract']

### Possible additional tests

In [ ]:
# Jaccard/Cosine similarity -> for query search
# WordNet-word-similarity -> for query search

## Add extracted data to raw data

#### Extract keywords from the abstract and add them to raw data

In [ ]:
# Keyword extraction
NLP = utils.NLP_spacy()
keywords_dataset = NLP.extract_refined_keywords(database, use_rake=True, refine_keywords=True,
                                                 column='abstract', keyword_length=3, num_keywords=15)

# Add the extracted keywords to the raw data dataframe
def join_keywords(keywords_list):
    keywords = ', '.join(kw for kw in keywords_list)
    return keywords
database_raw['keywords_nlp'] = list(map(join_keywords, keywords_dataset))
# database_raw['keywords_rake'] = list(map(join_keywords, keywords_dataset))          

#### Summarize the abstracts with a SBert model and add them to the raw data

In [ ]:
summaries = NLP.summarize_texts(database, column='abstract')

In [ ]:
len(summaries)
database_raw['summary'] = summaries

#### Detect and add the dataset's language based on titles

In [ ]:
language_dict = {'english':('EN', 'ENG'), 'french':('FR','FRA'), 'german':('DE','DEU'), 'italian':('IT','ITA'), 'not_found':('NA','NAN')}
database_raw['lang_3'] = database_raw.apply(lambda row: language_dict[utils.detect_language(row['title'], not_found=True)][1], axis=1)
database_raw['lang_2'] = database_raw.apply(lambda row: language_dict[utils.detect_language(row['title'], not_found=True)][0], axis=1)

#### Check metadata quality and save it in the database

In [ ]:
database_raw = utils.check_metadata_quality(database_raw, search_word='nan',
                                            search_columns=['abstract', 'keywords', 'metadata'],
                                            case_sensitive=False)

#### Clean the dataset for redis

In [ ]:
# clean up the str in the dataframe
database_raw = database_raw.replace(to_replace="'", value="-", regex=True)
database_raw = database_raw.replace(to_replace='\"', value="-", regex=True)
database_raw = database_raw.replace(to_replace="  ", value = " ", regex=True)
database_raw = database_raw.replace(to_replace="    ", value = " ", regex=True)

In [ ]:
# database_raw.to_csv('rawdata_scraper.csv')
database_raw.to_pickle('data/test_preprocessed.pkl')

In [ ]:
db = pd.read_pickle('data/test_preprocessed.pkl')
row = 201
print(db['abstract'].loc[row])
print(db['keywords_rake'].loc[row])
print(db['keywords_nlp'].loc[row])

#### Translate dataset

In [ ]:
old_data = pd.read_pickle("data/merged_data.pkl")
# new_db = pd.read_csv("data/geoservices_CH.csv", low_memory=False)
# new_db = new_db.fillna("nan")
# new_db = new_db.replace(to_replace="'", value="-", regex=True)
# new_db = new_db.replace(to_replace='\"', value="-", regex=True)
# new_db = new_db.replace(to_replace="  ", value = " ", regex=True)
# new_db = new_db.replace(to_replace="    ", value = " ", regex=True)

In [ ]:
from time import time
from tqdm.notebook import tqdm
tqdm.pandas()
translate_columns = ["keywords_nlp"]
for translate_column in translate_columns:
    for lang in ['en','fr','it']:
        tlang = time()
        new_col = translate_column+'_'+lang
        if translate_column == 'title':
            db[new_col] = db.progress_apply(lambda row: utils.translate_text(
                row[translate_column],to_lang=lang, from_lang=row['lang_3'],
                use_api=False), axis=1)
        elif translate_column == 'abstract':
            db[new_col] = db.progress_apply(lambda row: utils.translate_abstract(
                row[translate_column], to_lang=lang, from_lang=row['lang_3'],
                use_api=False), axis=1)
        elif translate_column == 'keywords':
            db[new_col] = db.progress_apply(lambda row: utils.translate_keywords(
                row[translate_column], to_lang=lang, from_lang=row['lang_3'],
                use_api=False), axis=1)
        elif translate_column == 'keywords_nlp':
            db[new_col] = db.progress_apply(lambda row: utils.translate_keywords(
                row[translate_column].split(','), to_lang=lang, from_lang=row['lang_3'],
                use_api=True), axis=1)
        else:
            print(f"Column {translate_column} could not be translated")
        print(f"Transaltion of {translate_column} in {lang} required {(time()-tlang)/60} minutes")

In [ ]:
from time import time
from tqdm.notebook import tqdm
tqdm.pandas()
translate_columns = ["abstract"]
for translate_column in translate_columns:
    for lang in ['it']:
        tlang = time()
        new_col = translate_column+'_'+lang
        if translate_column == 'title':
            db[new_col] = db.progress_apply(lambda row: utils.translate_text(
                row[translate_column],to_lang=lang, from_lang=row['lang_3'],
                use_api=False), axis=1)
        elif translate_column == 'abstract':
            one_piece = ' | '.join(db.loc[0:10]['abstract'].to_list()).replace("| nan |", "")
            abstract_translated = utils.translate_abstract(one_piece,
                                                           to_lang=lang, from_lang='DEU',
                                                           use_api=True)
            db[new_col] = abstract_translated.split(' | ')
            # db[new_col] = db.progress_apply(lambda row: utils.translate_abstract(
            #     row[translate_column], to_lang=lang, from_lang=row['lang_3'],
            #     use_api=True), axis=1)
        elif translate_column == 'keywords':
            db[new_col] = db.progress_apply(lambda row: utils.translate_keywords(
                row[translate_column], to_lang=lang, from_lang=row['lang_3'],
                use_api=False), axis=1)
        elif translate_column == 'keywords_nlp':
            db[new_col] = db.progress_apply(lambda row: utils.translate_keywords(
                row[translate_column].split(','), to_lang=lang, from_lang=row['lang_3'],
                use_api=True), axis=1)
        else:
            print(f"Column {translate_column} could not be translated")
        print(f"Transaltion of {translate_column} in {lang} required {(time()-tlang)/60} minutes")

In [ ]:
len(one_piece)

In [ ]:
db.columns

In [ ]:
print(db.loc[10000:10200][['abstract','abstract_it']].to_markdown())

In [ ]:
print(f"Abstract DE: {int(1280/60)} minutes")
print(f"Abstract IT: {int(5750/60)} minutes")
print(f"Abstract FR: {int(5092/60)} minutes")
print(f"Abstract EN: {int(5141/60)} minutes")

print(f"Title DE: {int(3738/60)} minutes")
print(f"Title IT: {int(13998/60)} minutes")
print(f"Title FR: {int(11284/60)} minutes")
print(f"Title EN: {int(17649/60)} minutes")

print(f"Keywords DE: {int(150/60)} minutes")
print(f"Keywords IT: {int(1065/60)} minutes")
print(f"Keywords FR: {int(1623/60)} minutes")
print(f"Keywords EN: {int(1384/60)} minutes")

print(f"Keywords_nlp DE: {int(4999/60)} minutes")
print(f"Keywords_nlp IT: {int(16972/60)} minutes")
print(f"Keywords_nlp FR: {int(19767/60)} minutes")
print(f"Keywords_nlp EN: {int(24484/60)} minutes")

print(f"\nTotal ET: {(3738/60+13998/60+11284/60+17649/60+3738/60+13998/60+11284/60+17649/60+150/60+1065/60+1623/60+1384/60+4999/60+16972/60+19767/60+24484/60)/60} hours")

In [ ]:
# transofrmers
# sentencepiece
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt") #  google/mt5-large

In [ ]:
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt", src_lang="en_XX")

In [ ]:
tokenizer.lang_code_to_id["de_DE"]

In [ ]:
t = "The quick brown fox jumps over a lazy dog"
model_in = tokenizer(t, return_tensors="pt")
translated_tensors = model.generate(**model_in, forced_bos_token_id=tokenizer.lang_code_to_id["de_DE"])
translated_tokens = tokenizer.batch_decode(translated_tensors, skip_special_tokens=True)

In [ ]:
" ".join(translated_tokens)

In [ ]:
NLP = utils.NLP_spacy()# missing keywords from abstract: 4803
keywords_dataset = NLP.extract_refined_keywords(db[(db['keywords_nlp']=='')],
                                                use_rake=True, refine_keywords=True,
                                                column='title', keyword_length=3, num_keywords=15)
k = 0
for i, row in db[(db['keywords_nlp']=='')].iterrows():
    if keywords_dataset[k]:
        db.loc[i, 'keywords_nlp'] = ",".join(keywords_dataset[k])
    k += 1

In [ ]:
pickle_path = "C:/Users/ma1021525/OneDrive - FHNW/Documents/Projects/git/Geoharvester/scraper/data/merged_data.pkl"
db = pd.read_pickle(pickle_path)
# utils.translate(text, to_lang='de', translator='google')

In [ ]:
db.columns

In [ ]:
print(db[['provider','title','endpoint','metadata']][1500:1520].to_markdown())

In [ ]:
for i, row in db[(db['lang_2']=='NA')&(db['provider']=='Bund')].iterrows():
    db.loc[i, 'lang_2'] = 'DE'
    db.loc[i, 'lang_3'] = 'DEU'

In [ ]:
language_dict = {'english':('EN', 'ENG'), 'french':('FR','FRA'), 'german':('DE','DEU'), 'italian':('IT','ITA'), 'not_found':('NA','NAN')}
for i, row in db[(db['lang_2']=='NA')&(db['keywords']!='nan')].iterrows():
    lang = utils.detect_language(row['keywords'], not_found=True)
    if lang[0] != 'not_found':
        lan = language_dict[lang]
        db.loc[i, 'lang_2'] = lan[0]
        db.loc[i, 'lang_3'] = lan[1]

In [ ]:
from deep_translator import GoogleTranslator, exceptions

def translate_text(text, to_lang, from_lang):
    language_dict = {'ENG':'en', 'FRA':'fr', 'DEU':'de', 'ITA':'it','NAN':'na'}
    if language_dict[from_lang] == to_lang:
        return text
    else:
        try:
            trnd = GoogleTranslator(source='auto', target=to_lang).translate(text.replace('_',' '))
        except exceptions.TranslationNotFound:
            trnd = 'nan'
        return trnd

def translate_abstract(text, to_lang, from_lang):
    language_dict = {'ENG':'en', 'FRA':'fr', 'DEU':'de', 'ITA':'it','NAN':'na'}
    if to_lang != language_dict[from_lang] and text != 'nan':
        if not text.startswith('http') or text.startswith('Link zu Metadaten:'):
            try:
                trnd = GoogleTranslator(source='auto', target=to_lang).translate(text.replace('_',' '))
            except exceptions.TranslationNotFound:
                trnd = 'nan'
            return trnd
        else:
            return 'nan'
    else:
        return text
    
def translate_keywords(text, to_lang, from_lang):
    if type(text) == str:
        text = [text]
    kwds = []
    for kwd in text:
        language_dict = {'ENG':'en', 'FRA':'fr', 'DEU':'de', 'ITA':'it','NAN':'na'}
        if kwd != 'nan' and language_dict[from_lang] != to_lang:
            if not kwd.startswith('http') or kwd.startswith('Link zu Metadaten:'):
                try:
                    kwd_trnsd = GoogleTranslator(source='auto', target=to_lang).translate(kwd.replace('_',' '))
                    if not kwd_trnsd:
                        kwd_trnsd = 'nan'
                except exceptions.TranslationNotFound:
                    kwd_trnsd = 'nan'
            else:
                kwd_trnsd = 'nan'
            kwds.append(kwd_trnsd)
        else:
            kwds.append(kwd)
    return ','.join(kwds)

In [ ]:
from tqdm import tqdm
tqdm.pandas()
languages = ['de']
translate_column = 'keywords_nlp'#'title'#'abstract' # title # keywords

for lang in languages:
    print(f"translating {translate_column} in {lang} ...")
    new_column = translate_column+'_'+lang
    if translate_column == 'title':
        db[new_column] = db.progress_apply(lambda row: translate_text(row[translate_column],to_lang=lang, from_lang=row['lang_3']), axis=1)
    elif translate_column == 'abstract':
        db[new_column] = db.progress_apply(lambda row: translate_abstract(row[translate_column], to_lang=lang, from_lang=row['lang_3']), axis=1)
    elif translate_column == 'keywords':
        db[new_column] = db.progress_apply(lambda row: translate_keywords(row[translate_column], to_lang=lang, from_lang=row['lang_3']), axis=1)
    else:
        db[new_column] = db.progress_apply(lambda row: translate_keywords(row[translate_column].split(','), to_lang=lang, from_lang=row['lang_3']), axis=1)
    db.to_pickle(pickle_path)

In [ ]:
column = 'keywords'
def count_links(field):
    if field.startswith('http') or field.startswith('Link zu Metadaten:'):
        return 1
    else:
        return 0
def count_nan(field):
    if field == 'nan':
        return 1
    else:
        return 0

db[f'{column}_link'] = db.progress_apply(lambda row: count_links(row[column]), axis=1)
db[f'{column}_nan'] = db.progress_apply(lambda row: count_nan(row[column]), axis=1)

In [ ]:
db_gr = db.groupby(['provider'], numeric_only=True).sum()[['abstract_link', 'abstract_nan','keywords_link','keywords_nan']]
db_gr.rename({'Geodienste':'Geodnst'}, inplace=True)
db_gr.sort_values(by=['provider'], ascending=False, inplace=True)
db_gr['count'] = db.groupby(['provider']).count()[['title']].sort_values(by=['provider'], ascending=False)['title'].to_list()
db_gr['abstract'] = 100 / db_gr['TITLE'] * db_gr['ABSTRACT']

In [ ]:
plt.figure(figsize = (5,5))
plt.barh(db_gr.index, 100 / db_gr['count'] * db_gr['abstract_link'])
plt.title("Abstracts in % containing just a link", fontsize=16)
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.rcParams.update({"font.size":10})
plt.xlim(0,100)
plt.ylabel("Providers")
plt.show()

In [ ]:
plt.figure(figsize = (5,5))
plt.barh(db_gr.index, 100 / db_gr['count'] * db_gr['abstract_nan'])
plt.title("Empty abstracts in %", fontsize=16)
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.rcParams.update({"font.size":10})
plt.xlim(0,100)
plt.ylabel("Providers")
plt.show()

In [ ]:
plt.figure(figsize = (5,5))
plt.barh(db_gr.index, 100 / db_gr['count'] * (db_gr['abstract_link']+db_gr['abstract_nan']))
plt.title("Unusable abstracts in %", fontsize=16)
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.rcParams.update({"font.size":10})
plt.xlim(0,100)
plt.ylabel("Providers")
plt.show()

In [ ]:
plt.figure(figsize = (5,5))
plt.barh(db_gr.index, 100 / db_gr['count'] * db_gr['keywords_link'])
plt.title("Keywords in % containing just a link", fontsize=16)
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.rcParams.update({"font.size":10})
plt.xlim(0,100)
plt.ylabel("Providers")
plt.show()

In [ ]:
plt.figure(figsize = (5,5))
plt.barh(db_gr.index, 100 / db_gr['count'] * db_gr['keywords_nan'])
plt.title("Empty keywords in %", fontsize=16)
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.rcParams.update({"font.size":10})
plt.xlim(0,100)
plt.ylabel("Providers")
plt.show()

In [ ]:
plt.figure(figsize = (5,5))
plt.barh(db_gr.index, 100 / db_gr['count'] * (db_gr['keywords_link']+db_gr['keywords_nan']))
plt.title("Unusable keywords in %", fontsize=16)
plt.autoscale(tight=True)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.tick_params(axis='y', which='both', left=False)
plt.rcParams.update({"font.size":10})
plt.xlim(0,100)
plt.ylabel("Providers")
plt.show()

<img src="https://c8j9w8r3.rocketcdn.me/wp-content/uploads/2017/03/join-types-merge-names.jpg">

## Evaluation ranking function
- Results relevance evaluation
- Results ranking evaluation

In [ ]:
import pandas as pd
import numpy as np
import sys, os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'server'))
from app.redis.methods import results_ranking, test_ranking

db = pd.read_pickle('data/preprocessed_data.pkl')
# db = db.rename(columns={'OWNER':'provider','TITLE':'title', 'NAME':'name','MAPGEO':'mapgeo','TREE':'tree','GROUP':'group',
#                    'ABSTRACT':'abstract','KEYWORDS':'keywords','KEYWORDS_NLP':'keywords_nlp','LEGEND':'legend',
#                    'CONTACT':'contact','ENDPOINT':'endpoint','METADATA':'metadata','UPDATE':'update','SERVICE':'service',
#                    'MAX_ZOOM':'max_zoom','CENTER_LAT':'center_lat','CENTER_LON':'center_lon','BBOX':'bbox','SUMMARY':'summary',
#                    'LANG_3':'lang_3','LANG_2':'lang_2','METAQUALITY':'metaquality'})
# db = db.rename(columns={'SERVICETYPE':'servicetype','SERVICELINK':'servicelink'})
# db.to_pickle('data/preprocessed_data.pkl')
db = db[['provider','title','abstract','keywords','keywords_nlp','metaquality']]

In [ ]:
# t0 = time()
db[(db['title'].str.contains('Winterthur', case=False)) | (db['keywords_nlp'].str.contains('winterthur', case=False))]# possiamo indicizzarlo essendo solo 6 keywords (verkehrsanordnung , städte zürich, kanton zürich, winterthur, tempo30, begegnungszonen'), mentre invece l'abstract di 33 parole viene diffice e le keywords normali sono vuote! (dataset: 17705)
# print(round(time()-t0,2))

In [ ]:
# t0 = time()
db[(db['title'].str.contains('Winterthur'))]
# print(round(time()-t0,2))

In [ ]:
mun_ch_15K = pd.read_csv('./data/mun_ch_15K.csv', header=None)
mun_ch_15K.columns = ['municipality']
mun_ch_15K['OWS_discovered'] = 0
mun_ch_15K['OWS'] = 0
for mun in mun_ch_15K['municipality'].to_list():
    t = db[(db['title'].str.contains(mun, case=False)) | (db['keywords_nlp'].str.contains(mun, case=False))]
    mun_ch_15K.loc[mun_ch_15K['municipality']==mun, ['OWS']]=len(db[(db['title'].str.contains(mun, case=False))])
    ows = len(t[~t.index.isin(db[(db['title'].str.contains(mun, case=False))].index.to_list())])#.loc[5311]['abstract'] # Also in this case THUN we have 5729 that is not discovered 
    mun_ch_15K.loc[mun_ch_15K['municipality']==mun, ['OWS_discovered']]=ows

In [ ]:
import matplotlib.pyplot as plt
mun_ch_15K[mun_ch_15K['OWS_discovered']<200]['OWS_discovered'].mean()

In [ ]:
print(mun_ch_15K[(mun_ch_15K['OWS_discovered']<200)].to_markdown())

In [ ]:
db[(db['abstract'] != 'nan') &
   (db['abstract'] != 'n.a.')&
   (~db['abstract'].str.contains('@'))&
   (db['provider'] == 'Bund')]['keywords_nlp'].to_list()

In [ ]:
db.loc[453]['keywords_nlp']

In [ ]:
db.loc[453]['abstract']

In [ ]:
# function to build the query in the database called db
# combined_search use AND while not combined use OR between the words
# it returns a pandas dataframe
def build_query(query_list, query_fields, combined_search=True):
    query_str = []
    if len(query_list) == 1:
        query_word = query_list[0]
        for field in query_fields:
            query_str.append(f"(db['{field}'].str.contains('{query_word}', case=False))")
        query_str = '|'.join(query_str)
    else:
        if combined_search:
            operator = '&'
        else:
            operator = '|'
        for query_word in query_list:
            query_part = []
            for field in query_fields:
                if field == 'keywords_nlp':
                    query_word = query_word.lower()
                query_part.append(f"(db['{field}'].str.contains('{query_word}', case=False))")
            query_str.append('|'.join(query_part))
        query_str = operator.join(query_str)
    print(query_str)
    return eval(f"db[{query_str}]")

def evaluate_search_results(sorted_results, test_index):
    scores = []
    for idx in test_index:
        if idx in sorted_results.index.to_list():
            scores.append(sorted_results.index.to_list().index(idx))
        else:
            scores.append(100)
    return scores

- Test 1: query_words = ['drohne','einschränkung']; test_rows = [20263, 20806, 23296]
- Test 2: query_words = ['durchlässigkeit','Deckschichten']; test_rows = [536, 537]
- Test 3: query_words = ['Amphibienzugstelle','fotopunkte']; test_rows = [634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644]
- Test 4: query_words = ['radweg','zürich']; test_rows = [17557]
- Test 4b: query_words = ['radweg','kanton schwyz']; test_rows = [11719, 12365]
- Test 4c: query_words = ['radweg','schweiz']; test_rows = [19936, 21190]
- Test 4d: query_words = ['veloweg','schweiz']; test_rows = [19936, 21190]
- Test 5: query_words = ['berufsinformationszentren','kanton bern']; test_rows = [5347,5348,5349,5350]
- Test 6: query_words = ['roemisch','pfosten','augusta raurica']; test_rows = [1781]
- Test 7: query_words = ['wildtierkorridore','schweiz']; test_rows = [19977, 20939, 11829, 12499]
- Test 7b: query_words = ['wildtierkorridore','kanton solothurn']; test_rows = [10398]
- Test 8: query_words = ['Abteilung Wasserbau','Bewilligung', 'Zürich']; test_rows = [17757]
- Test 9: query_words = ['bezirke','kanton zürich']; test_rows = [17764, 17863, 36651]
- Test 10: query_words = ['rohstoffe','kanton schaffhausen']; test_rows = [10321]
- Test 10b: query_words = ['rohstoffe','schweiz']; test_rows = [20701, 20702, 20698, 20700, 20412,20696,20699,20411]
- Test 11: query_words = ['eignung','solarenergie','schweiz']; test_rows = [20289, 20790, 20290, 20791]
- Test 11b: query_words = ['eignung','solarenergie','kanton aargau']; test_rows = [364, 365]

In [ ]:
query_words = ['CH-151.1']
test_rows = [634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644]
query_fields = ['title','keywords','keywords_nlp', 'abstract'] # all possible results
print(build_query(query_words,query_fields,combined_search=True).to_markdown())

In [ ]:
sorting_fields = ["title", "keywords"]# first list exact match, second list contains
sum_score = True

if sum_score:
    for i in range(len(sorting_fields)):
        test_db = test_ranking(build_query(query_words,query_fields,combined_search=True), query_words, [[],sorting_fields[0:i+1]])
        print(np.asarray(evaluate_search_results(test_db, test_rows)).sum(), sorting_fields[0:i+1])
    test_db = test_ranking(build_query(query_words,query_fields,combined_search=True), query_words, [[],['keywords']])
    print(np.asarray(evaluate_search_results(test_db, test_rows)).sum(), 'keywords')
    test_db = test_ranking(build_query(query_words,query_fields,combined_search=True), query_words)
    print(np.asarray(evaluate_search_results(test_db, test_rows)).sum(), 'custom GeoHarvester')
else:
    for i in range(len(sorting_fields)):
        test_db = test_ranking(build_query(query_words,query_fields,combined_search=True), query_words, [[],sorting_fields[0:i+1]])
        print(evaluate_search_results(test_db, test_rows), sorting_fields[0:i+1])
    test_db = test_ranking(build_query(query_words,query_fields,combined_search=True), query_words, [[],['keywords']])
    print(evaluate_search_results(test_db, test_rows), 'keywords')
    test_db = test_ranking(build_query(query_words,query_fields,combined_search=True), query_words)
    print(evaluate_search_results(test_db, test_rows), 'custom GeoHarvester')
print(f"Number of test rows: {len(test_rows)}")

In [ ]:
import numpy as np
np.asarray([12, 9, 16, 8, 1, 17, 0, 14, 11, 13, 10]).sum(), np.asarray([10, 3, 8, 7, 0, 6]).sum()

In [ ]:
print(test_db[0:100].to_markdown())

# TESTING

In [72]:
from cog.torque import Graph #CogDB
g = Graph("translation", cog_home="knowledge_graph")
g.put("carta dei pericoli","means","Gefahrenkarte")
g.put("carte des dangers", "means","Gefahrenkarte")
g.put("risk map", "means","Gefahrenkarte")
g.put("Gefahrenkarte", "lang", "german")
g.put("carta dei pericoli", "lang", "italian")
g.put("carte des dangers", "lang", "french")
g.put("risk map", "lang", "english")
# g.put("carta dei pericoli", "means", "Gefahrenkarte")
# g.put("risk map", "synonym","hazard map")

In [15]:
g.drop("Gefahrenkarte", "lang", "german")

In [ ]:
g.drop("risk map")

In [ ]:
# filter the edges in out()
g.v().tag("from").out().tag("to").view("test").render()

In [84]:
# all incoming edges for vertex
g.v(vertex="italian").inc().all()

{'result': [{'id': 'carta dei pericoli'}]}

In [78]:
l = []
if not l:
    print('OK')

OK


In [97]:
# all outgoing edges for vertex 
g.v(vertex="Gefahrenkarte").inc("means").has('lang','french').all()['result']

[{'id': 'carte des dangers'}]

In [66]:
# all vertex with specifi edge-vertex condition outgoing
g.v().has("lang","french").all()

{'result': [{'id': 'carte des dangers'}]}

In [76]:
[k['id'] for k in g.v().filter(func=lambda x: x.startswith("italian")).out().all()['result']]

['english', 'Gefahrenkarte']

In [31]:
for result in g.v("Gefahrenkarte").out().all()['result']:
    print(result['id'])
    if len(g.v(result['id']).out().all()['result']) > 0:
        for res in g.v(result['id']).out().all()['result']:
            print(" ",res['id'])